In [1]:
import pandas as pd
import numpy as np

In [2]:


## function 1 method that takes as finds and counts unique occurrances of a given column within a csv file
# @param[in] infilename: name of csv file
# @param[in] col: name of column of interest
# @param[in] outFile: exported file showing unique occurances in column "col" as well as how many times they have occured
def exportUniqueValuesInColumn(infilename, col,outFile):   
    df = pd.read_csv(infilename, encoding='latin-1',dtype="string")
    dfgroup = df.groupby(col).size() #df[col].unique()
    dfgroup.to_csv(outFile)
    print ("EXIT SUCCESS")

## functions 2 finds number of occurances of tree species per plot 
# @param[in] inCSVPlotsFile: name of csv file containing plot data
# @param[in] colPlots: name of column containing the IDs of the plots
# @parma[in] colSpecies: name of column containing the type of species
# @param[in] outFile: name of csv file exported with dominant species per plot
def exportPlotsWithDominantSpecies(inCsvPlotsFile, colPlots, colSpecies, outFile):
    #labelsGeneral = ["Pinus","Quercus","Fagus","Eucalyptus","Castanea","Juniperus","Betula","Populus","Fraxinus","Alnus","Abies","Myrica","Arbutus","Acer","Ilex","Salix","Pseudotsuga","Laurus","Crataegus","Sorbus","Corylus","Persea","Acacia","Larix","Prunus","Chamaecyparis","Olea","Tilia","Robinia","Platanus","Cupressus","Picea","Ulmus","Cedrus","Juglans","Phillyrea","Pyrus","Taxus","Otras","Phoenix","Malus","Otros","Sambucus","Tamarix","Celtis","Picconia","Ficus","Apollonias","Heberdenia","Ceratonia","Pinus","Quercus","Fagus","Eucalyptus","Castanea","Juniperus","Betula","Populus","Fraxinus","Alnus","Abies","Myrica","Arbutus","Acer","Ilex","Salix","Pseudotsuga","Laurus","Crataegus","Sorbus","Corylus","Persea","Acacia","Larix","Prunus","Chamaecyparis","Olea","Tilia","Robinia","Platanus","Cupressus","Picea","Ulmus","Cedrus","Juglans","Phillyrea","Pyrus","Taxus","Otras","Phoenix","Malus","Otros","Sambucus","Tamarix","Celtis","Picconia","Ficus","Apollonias","Heberdenia","Ceratonia"]
    df = pd.read_csv(inCsvPlotsFile, encoding='latin-1', dtype="string")
    df_counts = df.groupby([colPlots, colSpecies]).size().reset_index(name='Count')
    df_pivot = df_counts.pivot(index=colPlots, columns=colSpecies, values='Count').fillna(0)
    #df_pivot = df_pivot[labelsGeneral]
    df_pivot.reset_index(inplace=True)
    #df_pivot = df_pivot[labelsGeneral]
    df_pivot.to_csv(outFile, index=False)
    print("File saved in: ", outFile)


## function 3 takes as input the output of function 2 and adds a column name "sum", which
#  contains the sum of trees per plot and then calculates the percentage of each specie/genera per plot
def getPercentageOfSpeciesPerPlot(inCsv, outCsv):
    df = pd.read_csv(inCsv, encoding='latin-1', low_memory=False)
    labels = list(df.columns)
    print(labels)
    count = 1
    df['sumOfAllTrees'] = df[df.columns[1:]].sum(axis=1)
    """
    species_columns = df.columns[1:]  # Assuming the first column is 'PlotID'
    for species in species_columns:
        percentage_column_name = f'{species}_per'
        df[percentage_column_name] = df[species] / df['sumOfAllTrees']
    
    df.to_csv(outCsv, index=False)
    """
    df_copy = df.copy()
    species_columns = df_copy.columns[1:]
    for species in species_columns:
        df_copy[species] = df[species] / df_copy['sumOfAllTrees']

       
    df_copy.to_csv(outCsv, index=False)

    print("   *** getPercentageOfSpeciesPerPlot: exit success!   ***")


## function 4 takes as input the output of function 3 and creates a folder for each class
#  within the folder it stores three files: (1) all the plots that contain only this class,
#  (2) all the plots that contain 75% and more than the specified class but less than 100%
#  (3) all the plots that contain 50-75% of this class
#  param[in] inCsv the output of Function 3 containing the how many of each tree class exist in each plot in percentages
#  param[in] outDir the directory where the new folders will be stored
def getPlotIDsPerClass(inCsv, colPlots, outDir):
    df = pd.read_csv(inCsv, encoding='latin-1', low_memory=False)
    if 'sumOfAllTrees' in df.columns:
        df = df.drop(columns=['sumOfAllTrees'])

    df.set_index(colPlots, inplace=True)
    for species_column in df.columns:
        filtered_df100 = df[df[species_column] >= 0.99999999999]
        filtered_df75  = df[df[species_column] >= 0.75 ]
        filtered_df50  = df[df[species_column] >= 0.5  ]
        selected_plots100 = filtered_df100.index.tolist()
        selected_plots75  = filtered_df75.index.tolist()
        selected_plots50  = filtered_df50.index.tolist()
        if (selected_plots75!=[]):
            print(f"Plots with more than 75% of {species_column}: {selected_plots75}")

    

def exportPlotsWithDominantSpecies2(inCsvPlotsFile, colPlots, colSpecies, outFile, top_n=5):
    # Read CSV file into a DataFrame
    df = pd.read_csv(inCsvPlotsFile, encoding='latin-1', dtype="string")
    
    # Group by plot and species, calculate counts
    df_counts = df.groupby([colPlots, colSpecies]).size().reset_index(name='Count')
    
    # Pivot the table
    df_pivot = df_counts.pivot(index=colPlots, columns=colSpecies, values='Count').fillna(0)
    df_pivot.reset_index(inplace=True)
    
    # Add columns for dominant species and their counts
    df_pivot['Dominant_Species'] = df_pivot.iloc[:, 1:].apply(lambda x: x.nlargest(top_n).index.tolist(), axis=1)
    df_pivot['Dominant_Count'] = df_pivot.iloc[:, 1:].apply(lambda x: x.nlargest(top_n).tolist(), axis=1)
    
    # Explode the lists to separate rows for each dominant species
    df_exploded = df_pivot.explode('Dominant_Species')
    df_exploded['Dominant_Count'] = df_exploded['Dominant_Count'].astype(int)
    
    # Sort the DataFrame by plot ID and dominant count
    df_exploded = df_exploded.sort_values(by=[colPlots, 'Dominant_Count'], ascending=[True, False])
    
    # Transpose the DataFrame
    df_transposed = df_exploded.set_index(['Dominant_Species', colPlots]).unstack().transpose().reset_index()
    
    # Rename the columns
    df_transposed.columns = [colPlots, 'Dominant_Species', 'Dominant_Count']
    
    # Export the result to a CSV file
    df_transposed.to_csv(outFile, columns=[colPlots, 'Dominant_Species', 'Dominant_Count'], index=False)





def exportPlotsWithDominantSpeciesAndSelectMostCommonSpecies(inCsvPlotsFile, colPlots, outFile):
    df = pd.read_csv(inCsvPlotsFile, encoding='latin-1', dtype="string")
    column_labels = df.columns.tolist()

    broaderList      = [colPlots, "Pinus", "Quercus", "Fagus", "Eucalyptus", "Castanea", "Juniperus", "Betula", "Populus", "Fraxinus", "Alnus", "Abies", "Myrica", "Arbutus"] 
    mostDominantList =[colPlots, "Pinus", "Quercus", "Fagus", "Eucalyptus", "Castanea", "Juniperus", "Betula"] 


    broaderListFiltered = [item for item in broaderList if item in column_labels]
    print(broaderListFiltered)
    mostDominantListFiltered = [item for item in mostDominantList if item in column_labels]

   
    df_pivot = df[broaderList]
    zero_rows = (df_pivot.iloc[:,1:]==0).all(axis=1)
    df_pivot = df_pivot[~zero_rows]
    df_pivot.to_csv(outFile+"broaderList.csv", index=False)

    df_pivot = df[mostDominantList]
    zero_rows = (df_pivot.iloc[:,1:]==0).all(axis=1)
    df_pivot = df_pivot[~zero_rows]
    df_pivot.to_csv(outFile+"broaderMostDominant.csv", index=False)

def exportTop3SpeciesWithPercentage(inCsvPlotsFile, colPlots, colSpecies, outFile):
    df = pd.read_csv(inCsvPlotsFile, encoding='latin-1', low_memory=False)
    column_labels = df.columns.tolist()
    plots = df[colPlots].to_list()
    dfValues = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')    
    #dfValues[colPlots] = plots
    

    df_pivot['Dominant_Species'] = df_pivot.iloc[:, 1:].apply(lambda x: x.nlargest(top_n).index.tolist(), axis=1)
    df_pivot['Dominant_Count']   = df_pivot.iloc[:, 1:].apply(lambda x: x.nlargest(top_n).tolist(), axis=1)
      # Explode the lists to separate rows for each dominant species
    df_exploded = df_pivot.explode('Dominant_Species')
    df_exploded['Dominant_Count'] = df_exploded['Dominant_Count'].astype(int)
    
    # Sort the DataFrame by plot ID and dominant count
    df_exploded = df_exploded.sort_values(by=[colPlots, 'Dominant_Count'], ascending=[True, False])
    
    df_transposed = df_exploded.set_index(['Dominant_Species', colPlots]).unstack().transpose().reset_index()
    
    # Rename the columns
    df_transposed.columns = [colPlots, 'Dominant_Species', 'Dominant_Count']
    
    # Export the result to a CSV file
    df_transposed.to_csv(outFile, columns=[colPlots, 'Dominant_Species', 'Dominant_Count'], index=False)


def exportPlotsWithDominantSpecies3(inCsvPlotsFile, colPlots, colSpecies, outFile, top_n=5):
    # Read CSV file into a DataFrame
    df = pd.read_csv(inCsvPlotsFile, encoding='latin-1', low_memory=False)
    
    # Group by plot and species, calculate counts
    df_counts = df.groupby([colPlots, colSpecies]).size().reset_index(name='Count')
    
    # Pivot the table
    df_pivot = df_counts.pivot(index=colPlots, columns=colSpecies, values='Count').fillna(0)
    df_pivot.reset_index(inplace=True)
    
    # Convert counts to numeric (important step)
    df_pivot.iloc[:, 1:] = df_pivot.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    
    # Add columns for dominant species and their counts
    df_pivot['Dominant_Species'] = df_pivot.iloc[:, 1:].apply(lambda x: x.nlargest(top_n).index.tolist(), axis=1)
    df_pivot['Dominant_Count'] = df_pivot.iloc[:, 1:].apply(lambda x: x.nlargest(top_n).tolist(), axis=1)
    
    # Explode the lists to separate rows for each dominant species
    df_exploded = df_pivot.explode('Dominant_Species')
    df_exploded['Dominant_Count'] = df_exploded['Dominant_Count'].astype(int)
    
    # Sort the DataFrame by plot ID and dominant count
    df_exploded = df_exploded.sort_values(by=[colPlots, 'Dominant_Count'], ascending=[True, False])
    
    # Transpose the DataFrame
    df_transposed = df_exploded.set_index(['Dominant_Species', colPlots]).unstack().transpose().reset_index()
    
    # Rename the columns
    df_transposed.columns = [colPlots, 'Dominant_Species', 'Dominant_Count']
    
    # Export the result to a CSV file
    df_transposed.to_csv(outFile, columns=[colPlots, 'Dominant_Species', 'Dominant_Count'], index=False)



def groupAndExportCountToCSV(dfinput, col1, col2, csv_filename):
    df = pd.read_csv(dfinput, encoding='latin-1', dtype="string")
    grouped = df.groupby([col1, col2])
    aggregated = grouped.size().reset_index(name='Count')
    aggregated.to_csv(csv_filename, index=False)


def rearrangeGroupsInColumns(inCSVFile, colPlots, colSpeciesAll, outFileThreeColsAll):
    df = pd.read_csv(inCSVFile)

    pivoted_df = df.pivot(index=colPlots, columns=colSpeciesAll, values='count').reset_index()

    pivoted_df = pivoted_df.fillna(0)

    # Rename the columns
    pivoted_df.columns.name = None  # Remove the column name created by the pivot
    pivoted_df.columns = [str(col) for col in pivoted_df.columns]  # Convert columns to strings
    pivoted_df = pivoted_df.rename(columns={colPlots: colPlots})

    # Save the rearranged DataFrame to a new CSV file
    pivoted_df.to_csv(outFileThreeColsAll, index=False)